# 84.51º
## Association Rule Mining

In retail data, knowing your customer is extremely because it allows
for digital personalization that enhances the customer experience.
At 84.51º, we take several different approaches to knowing our customer,
all of which involve an incredible amount of science.

In this workshop, we're going to cover a simple machine learning style algorithm
used to find cross shopping, or items that are frequently purchased together. In
order to do this, we're going to use Association Rule Mining.

### Definitions

**Association Rule** - Rule of the form $X \implies Y$, where $X$ and $Y$ are item sets. Note the implication notation does not mean $X$ causes $Y$, but rather $X$ co-occurs with $Y$ (e.g. $X$ and $Y$ are bought together).

**Item Set** - A collection of one or more items.

**K-Item Set** - An item set that contains $k$ items.

**Support Count** - Frequency of a particular item set

**Support** - Fraction of transactions that contain a particular item set. $\frac{X}{\mid T \mid}$, where $X$ is the number of times the item set appears and $\mid T \mid$ is the number of total transactions.

**Confidence** - Measures how often item in $Y$ appear in transactions that contain $X$.

Let's start with a universe of some items. This means all of our transactions will be a subset of this particular set.

In [3]:
items = {'milk', 'bread', 'cheese', 'eggs', 'soup', 'beer', 'cookies', 'crackers'}

Now, let's list some transactions, which consist of multiple items.

In [13]:
transactions = [
    {'milk', 'eggs'},
    {'beer', 'cheese', 'bread'},
    {'soup', 'cheese', 'bread', 'eggs', 'crackers'},
    {'cookies', 'soup', 'milk'},
    {'milk', 'eggs', 'cheese', 'bread'},
    {'milk', 'eggs', 'cookies'},
    {'cheese', 'beer', 'bread'},
    {'soup', 'crackers', 'bread'},
    {'bread', 'cheese', 'eggs'}
]

Finally, the implementation of `support` and `confidence`:

In [4]:
def support_count(item_set, transactions):
    count = 0
    for transaction in transactions:
        # The <= sign means subset or equal set
        if item_set <= transaction:
            count += 1
    return count

def support(item_set, transactions):
    sc = support_count(item_set, transactions)
    return sc / len(transactions)

def confidence(item_set_x, item_set_y, transactions):
    x_u_y = item_set_x.union(item_set_y)
    return support_count(x_u_y, transactions) / support_count(item_set_x, transactions)

In order to compute these values for every combination, we have to know every possible combination of subsets.

Here's all the possible combinations for the 8 items available for baskets:

In [5]:
import itertools

for i in range(1, len(items)): # i will be what length of item sets we're looking for.
    for combo in itertools.combinations(items, i):
        print(combo)


('cheese',)
('milk',)
('crackers',)
('cookies',)
('beer',)
('eggs',)
('bread',)
('soup',)
('cheese', 'milk')
('cheese', 'crackers')
('cheese', 'cookies')
('cheese', 'beer')
('cheese', 'eggs')
('cheese', 'bread')
('cheese', 'soup')
('milk', 'crackers')
('milk', 'cookies')
('milk', 'beer')
('milk', 'eggs')
('milk', 'bread')
('milk', 'soup')
('crackers', 'cookies')
('crackers', 'beer')
('crackers', 'eggs')
('crackers', 'bread')
('crackers', 'soup')
('cookies', 'beer')
('cookies', 'eggs')
('cookies', 'bread')
('cookies', 'soup')
('beer', 'eggs')
('beer', 'bread')
('beer', 'soup')
('eggs', 'bread')
('eggs', 'soup')
('bread', 'soup')
('cheese', 'milk', 'crackers')
('cheese', 'milk', 'cookies')
('cheese', 'milk', 'beer')
('cheese', 'milk', 'eggs')
('cheese', 'milk', 'bread')
('cheese', 'milk', 'soup')
('cheese', 'crackers', 'cookies')
('cheese', 'crackers', 'beer')
('cheese', 'crackers', 'eggs')
('cheese', 'crackers', 'bread')
('cheese', 'crackers', 'soup')
('cheese', 'cookies', 'beer')
('che

As you can imagine, computing frequent co-occurences of items can be prohibitively and computationally expensive, especially if you're working with a large number of products. At Kroger, we have a few more items than our simple example. (Does anyone have a guess as to how many unique UPCs we have?) Having that many UPCs would make this brute force approach take an inordinate amount of time. This is because the number of combinations grows at a rate of $2^n$, where $n$ is the number of available items. Even if we had just 1,000 items, the total number of item sets would overflow an unsigned 64 bit integer. (Which has a maximum value of 18,446,744,073,709,551,615, or 18 quintillion for those keeping track at home)

We can make the task significantly easier by finding rules with **support and confidence values greater than a threshold**. For example, we only care about items that are frequently purchased together, and don't care about pairs that aren't purchased frequently together.

# The Brute Force Way

In [18]:
import itertools

def combinations(items):
    for i in range(1, len(items)):
        for combo in itertools.combinations(items, i):
           yield combo
        
answers = []
checks = 0
for combo in combinations(items):
    checks += 1
    if support(set(combo), transactions) >= .5:
        answers.append(combo)

print('Checked ' + str(checks) + ' item sets.')
print('The ones with 50% or more?: ', answers)

Checked 254 item sets.
The ones with 50% or more?:  [('cheese',), ('eggs',), ('bread',), ('cheese', 'bread')]


# Do you know of any way that we can make this computation go a bit quicker, instead of computing it for all possible item sets?


## Any ideas?

Hint: Can a particular superset of an item set have a higher support than the item set itself?

---

This is called an **apriori** algorithim and is covered in many Data Science courses. It works by ignoring supersets whose subsets have a support lower than the threshold. **Why does this work?**

We're going to use some common data science libraries, Pandas and MLXtend in Python to do this so we don't have to write the implementation of the apriori algorithm.

In [16]:
# import necessary classes
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()

dataset = encoder.fit(transactions).transform(transactions)
frame = pd.DataFrame(dataset, columns=encoder.columns_)
print("The representation that mlxtend uses:")
print(frame)

The representation that mlxtend uses:
    beer  bread  cheese  cookies  crackers   eggs   milk   soup
0  False  False   False    False     False   True   True  False
1   True   True    True    False     False  False  False  False
2  False   True    True    False      True   True  False   True
3  False  False   False     True     False  False   True   True
4  False   True    True    False     False   True   True  False
5  False  False   False     True     False   True   True  False
6   True   True    True    False     False  False  False  False
7  False   True   False    False      True  False  False   True
8  False   True    True    False     False   True  False  False


In [17]:
from mlxtend.frequent_patterns import apriori

print(apriori(frame, min_support=0.5, use_colnames=True))

    support         itemsets
0  0.666667          (bread)
1  0.555556         (cheese)
2  0.555556           (eggs)
3  0.555556  (cheese, bread)
